## Overview

This notebook is based on a workshop tutorial originally given at FOCAPD in 2019.
It demonstrated the following capabilities:

- Construct a steady-state flowsheet using the IDAES unit model library
- Connecting unit models in a  flowsheet using Arcs
- Using the SequentialDecomposition tool to initialize a flowsheet with recycle
- Fomulate and solve an optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 

Changes were made to use the notebook to demonstrate use of the IDAES Data Management Framework (DMF):
- The underlying Python modules were adapted to use the DMF for loading property data
- The optimization problem was removed to shorten the example

## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups,. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

The flowsheet that we will be using for this module is shown below with the stream conditions. We will be processing toluene and hydrogen to produce at least 370 TPY of benzene. As shown in the flowsheet, there are two flash tanks, F101 to separate out the non-condensibles and F102 to further separate the benzene-toluene mixture to improve the benzene purity.  Note that typically a distillation column is required to obtain high purity benzene but that is beyond the scope of this workshop. The non-condensibles separated out in F101 will be partially recycled back to M101 and the rest will be either purged or combusted for power generation.We will assume ideal gas for this flowsheet. The properties required for this module are available in the same directory:

- hda_ideal_VLE.py
- hda_reaction.py

The state variables chosen for the property package are **flows of component by phase, temperature and pressure**. The components considered are: **toluene, hydrogen, benzene and methane**. Therefore, every stream has 8 flow variables, 1 temperature and 1 pressure variable. 

### Data Management Framework

In this version of the example, the DMF "workspace" is used to store these state variables. This involves the use of additional files:

- config.yaml - DMF Configuration
- resourcedb.json - The DMF's file-based datastore, where the values and their provenance are stored.


![](module_2_flowsheet.png)

Importing required pyomo and idaes components
-----------
To construct a flowsheet, we will need several components from the pyomo and idaes package. Let us first import the following components from Pyomo:
- Constraint (to write constraints)
- Var (to declare variables)
- ConcreteModel (to create the concrete model object)
- Expression (to evaluate values as a function of variables defined in the model)
- Objective (to define an objective function for optimization)
- SolverFactory (to solve the problem)
- TransformationFactory (to apply certain transformations)
- Arc (to connect two unit models)
- SequentialDecomposition (to initialize the flowsheet in a sequential mode)

For further details on these components, please refer to the pyomo documentation: https://pyomo.readthedocs.io/en/latest/


In [3]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.network import Arc, SequentialDecomposition

From idaes, we will be needing the FlowsheetBlock and the following unit models:
- Mixer
- Heater
- StoichiometricReactor
- <span style="color:blue">**Flash**</span>
- Separator (splitter) 
- PressureChanger

In [4]:
from idaes.core import FlowsheetBlock

In [5]:
from idaes.unit_models import (PressureChanger,
                               Mixer,
                               Separator as Splitter,
                               Heater,
                               StoichiometricReactor)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, import the remaining unit models highlighted in blue above and run the cell using `Shift+Enter` after typing in the code. 
</div>


In [6]:
from idaes.unit_models import Flash

We will also be needing some utility tools to put together the flowsheet and calculate the degrees of freedom. 

In [7]:
from idaes.unit_models.pressure_changer import ThermodynamicAssumption
from idaes.core.util.model_statistics import degrees_of_freedom

Importing required thermo and reaction package
-----------


The final set of imports are to import the thermo and reaction package for the HDA process. We have created a custom thermo package that assumes Ideal Gas with support for VLE. 

The reaction package here is very simple as we will be using only a StochiometricReactor and the reaction package consists of the stochiometric coefficients for the reaction and the parameter for the heat of reaction. 

Let us import the following modules and they are in the same directory as this jupyter notebook:
      <ul>
         <li>hda_ideal_VLE as thermo_props</li>
         <li>hda_reaction as reaction_props </li>
      </ul>
</div>

In [3]:
import hda_ideal_VLE as thermo_props
import hda_reaction as reaction_props

Constructing the Flowsheet
----------------------------------

We have now imported all the components, unit models, and property modules we need to construct a flowsheet. Let us create a ConcreteModel and add the flowsheet block as we did in module 1. 

In [9]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})

We now need to add the property packages to the flowsheet. Unlike Module 1, where we only had a thermo property package, for this flowsheet we will also need to add a reaction property package. 

In [10]:
m.fs.thermo_params = thermo_props.HDAParameterBlock()
m.fs.reaction_params = reaction_props.HDAReactionParameterBlock(
        default={"property_package": m.fs.thermo_params})

## Viewing imported properties with the DMF

Since the property package above used the DMF to load the constant values used for calculating property values, we can look at the provenance and data programmatically.

In [11]:
from idaes.dmf.resource import TidyUnitData as TDU
prop_name = 'dh_vap'
rsrc = m.fs.thermo_params.get_property_resource(prop_name)
print(f"Property {prop_name} source: {rsrc.formatted_source()}")
print(f"Property {prop_name} data:")
TDU(rsrc.data).table

Property dh_vap source: Reid, R. C., Prausnitz, J. M., & Poling, B. E. (1987). The properties of gases and liquids. ISBN: 9780070517998 Date: 1987
Property dh_vap data:


,compound,value
0,benzene,33870.0
1,toluene,38262.0
2,hydrogen,0.0
3,methane,0.0
4,diphenyl,62710.0


Adding Unit Models
-----

Let us start adding the unit models we have imported to the flowsheet. Here, we are adding the Mixer (assigned a name M101) and a Heater (assigned a name H101). Note that, all unit models need to be given a property package argument. In addition to that, there are several arguments depending on the unit model, please refer to the documentation for more details (https://idaes-pse.readthedocs.io/en/latest/models/index.html). For example, the Mixer unit model here is given a `list` consisting of names to the three inlets. 

In [12]:
m.fs.M101 = Mixer(default={"property_package": m.fs.thermo_params,
                           "inlet_list": ["toluene_feed", "hydrogen_feed", "vapor_recycle"]})

m.fs.H101 = Heater(default={"property_package": m.fs.thermo_params,
                            "has_pressure_change": False,
                            "has_phase_equilibrium": True})

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Let us now add the StoichiometricReactor(assign the name R101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"reaction_package": m.fs.reaction_params </li>
         <li>"has_heat_of_reaction": True </li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>
</div>

In [13]:
m.fs.R101 = StoichiometricReactor(
            default={"property_package": m.fs.thermo_params,
                     "reaction_package": m.fs.reaction_params,
                     "has_heat_of_reaction": True,
                     "has_heat_transfer": True,
                     "has_pressure_change": False})

Let us now add the Flash(assign the name F101) and pass the following arguments:
      <ul>
         <li>"property_package": m.fs.thermo_params</li>
         <li>"has_heat_transfer": True</li>
         <li>"has_pressure_change": False</li>
      </ul>

In [14]:
m.fs.F101 = Flash(default={"property_package": m.fs.thermo_params,
                               "has_heat_transfer": True,
                               "has_pressure_change": True})

Let us now add the Splitter(S101), PressureChanger(C101) and the second Flash(F102). 

In [15]:
m.fs.S101 = Splitter(default={"property_package": m.fs.thermo_params,
                               "ideal_separation": False,
                               "outlet_list": ["purge", "recycle"]})
    

m.fs.C101 = PressureChanger(default={
            "property_package": m.fs.thermo_params,
            "compressor": True,
            "thermodynamic_assumption": ThermodynamicAssumption.isothermal})
    
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

Connecting Unit Models using Arcs
-----
We have now added all the unit models we need to the flowsheet. However, we have not yet specifed how the units are to be connected. To do this, we will be using the `Arc` which is a pyomo component that takes in two arguments: `source` and `destination`. Let us connect the outlet of the mixer(M101) to the inlet of the heater(H101). 

In [16]:
m.fs.s03 = Arc(source=m.fs.M101.outlet, destination=m.fs.H101.inlet)


![](module_2_flowsheet.png) 

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Now, connect the H101 outlet to the R101 inlet using the cell above as a guide. 
</div>



In [17]:
m.fs.s04 = Arc(source=m.fs.H101.outlet, destination=m.fs.R101.inlet)

We will now be connecting the rest of the flowsheet as shown below. Notice how the outlet names are different for the flash tanks F101 and F102 as they have a vapor and a liquid outlet. 

In [18]:
m.fs.s05 = Arc(source=m.fs.R101.outlet, destination=m.fs.F101.inlet)
m.fs.s06 = Arc(source=m.fs.F101.vap_outlet, destination=m.fs.S101.inlet)
m.fs.s08 = Arc(source=m.fs.S101.recycle, destination=m.fs.C101.inlet)
m.fs.s09 = Arc(source=m.fs.C101.outlet,
               destination=m.fs.M101.vapor_recycle)
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)

We have now connected the unit model block using the arcs. However, each of these arcs link to ports on the two unit models that are connected. In this case, the ports consist of the state variables that need to be linked between the unit models. Pyomo provides a convenient method to write these equality constraints for us between two ports and this is done as follows:

In [19]:
TransformationFactory("network.expand_arcs").apply_to(m)

Adding expressions to compute purity and operating costs
---
In this section, we will add a few Expressions that allows us to evaluate the performance. Expressions provide a convenient way of calculating certain values that are a function of the variables defined in the model. For more details on Expressions, please refer to: https://pyomo.readthedocs.io/en/latest/pyomo_modeling_components/Expressions.html

For this flowsheet, we are interested in computing the purity of the product Benzene stream (i.e. the mole fraction) and the operating cost which is a sum of the cooling and heating cost. 

Let us first add an Expression to compute the mole fraction of benzene in the `vap_outlet` of F102 which is our product stream. Please note that the var flow_mol_phase_comp has the index - [time, phase, component]. As this is a steady-state flowsheet, the time index by default is 0. The valid phases are ["Liq", "Vap"]. Similarly the valid component list is ["benzene", "toluene", "hydrogen", "methane"].

In [20]:
m.fs.purity = Expression(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] /
        (m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
         + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

Now, let us add an expression to compute the cooling cost assuming a cost of 0.212E-4 $/kW. Note that cooling utility is required for the reactor (R101) and the first flash (F101). 

In [21]:
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-m.fs.F101.heat_duty[0]) +
                                   0.212e-7 * (-m.fs.R101.heat_duty[0]))


Now, let us add an expression to compute the heating cost assuming the utility cost as follows:
      <ul>
         <li>2.2E-4 dollars/kW for H101</li>
         <li>1.9E-4 dollars/kW for F102</li>
      </ul>
Note that the heat duty is in units of watt (J/s). 

In [22]:
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                   1.9e-7 * m.fs.F102.heat_duty[0])

Let us now add an expression to compute the total operating cost per year which is basically the sum of the cooling and heating cost we defined above. 

In [23]:
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                           (m.fs.heating_cost +
                                            m.fs.cooling_cost)))

Fixing feed conditions
---

Let us first check how many degrees of freedom exist for this flowsheet using the `degrees_of_freedom` tool we imported earlier. 

In [24]:
print(degrees_of_freedom(m))

49


We will now be fixing the toluene feed stream to the conditions shown in the flowsheet above. Please note that though this is a pure toluene feed, the remaining components are still assigned a very small non-zero value to help with convergence and initializing. 

In [25]:
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(0.30)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.toluene_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.toluene_feed.temperature.fix(303.2)
m.fs.M101.toluene_feed.pressure.fix(350000)


Similarly, let us fix the hydrogen feed to the following conditions in the next cell:
      <ul>
         <li>F<sub>H2</sub> = 0.30 mol/s</li>
         <li>F<sub>CH4</sub> = 0.02 mol/s</li>
         <li>Remaining components = 1e-5 mol/s</li>
         <li>T = 303.2 K</li>
         <li>P = 350000 Pa</li>
      </ul>



In [26]:
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "hydrogen"].fix(0.30)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Vap", "methane"].fix(0.02)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "benzene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "toluene"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "hydrogen"].fix(1e-5)
m.fs.M101.hydrogen_feed.flow_mol_phase_comp[0, "Liq", "methane"].fix(1e-5)
m.fs.M101.hydrogen_feed.temperature.fix(303.2)
m.fs.M101.hydrogen_feed.pressure.fix(350000)

Fixing unit model specifications
---

Now that we have fixed our inlet feed conditions, we will now be fixing the operating conditions for the unit models in the flowsheet. Let us set set the H101 outlet temperature to 600 K. 

In [27]:
m.fs.H101.outlet.temperature.fix(600)

For the StoichiometricReactor, we have to define the conversion in terms of toluene. This requires us to create a new variable for specifying the conversion and adding a Constraint that defines the conversion with respect to toluene. The second degree of freedom for the reactor is to define the heat duty. In this case, let us assume the reactor to be adiabatic i.e. Q = 0. 

In [28]:
m.fs.R101.conversion = Var(initialize=0.75, bounds=(0, 1))

m.fs.R101.conv_constraint = Constraint(
    expr=m.fs.R101.conversion*m.fs.R101.inlet.
    flow_mol_phase_comp[0, "Vap", "toluene"] ==
    (m.fs.R101.inlet.flow_mol_phase_comp[0, "Vap", "toluene"] -
     m.fs.R101.outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))

m.fs.R101.conversion.fix(0.75)
m.fs.R101.heat_duty.fix(0)

The Flash conditions for F101 can be set as follows. 

In [29]:
m.fs.F101.vap_outlet.temperature.fix(325.0)
m.fs.F101.deltaP.fix(0)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Set the conditions for Flash F102 to the following conditions:
      <ul>
         <li>T = 375 K</li>
         <li>deltaP = -200000</li>
      </ul>

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [30]:
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

Let us fix the purge split fraction to 20% and the outlet pressure of the compressor is set to 350000 Pa. 

In [31]:
m.fs.S101.split_fraction[0, "purge"].fix(0.2)
m.fs.C101.outlet.pressure.fix(350000)

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now defined all the feed conditions and the inputs required for the unit models. The system should now have 0 degrees of freedom i.e. should be a square problem. Please check that the degrees of freedom is 0. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>

In [32]:
print(degrees_of_freedom(m))

20


Initialization
------------------

This section will demonstrate how to use the built-in sequential decomposition tool to initialize our flowsheet.

![](module_2_flowsheet.png) 


Let us first create an object for the SequentialDecomposition and specify our options for this. 

In [33]:
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

Which is the tear stream? Display tear set and order

In [34]:
for o in heuristic_tear_set:
    print(o.name)

fs.s03


What sequence did the SD tool determine to solve this flowsheet with the least number of tears? 

In [35]:
for o in order:
    print(o[0].name)

fs.H101
fs.R101
fs.F101
fs.S101
fs.C101
fs.M101


 

![](module_2_tear_stream.png) 


The SequentialDecomposition tool has determined that the tear stream is the mixer outlet. We will need to provide a reasonable guess for this.

In [36]:
tear_guesses = {
        "flow_mol_phase_comp": {
                (0, "Vap", "benzene"): 1e-5,
                (0, "Vap", "toluene"): 1e-5,
                (0, "Vap", "hydrogen"): 0.30,
                (0, "Vap", "methane"): 0.02,
                (0, "Liq", "benzene"): 1e-5,
                (0, "Liq", "toluene"): 0.30,
                (0, "Liq", "hydrogen"): 1e-5,
                (0, "Liq", "methane"): 1e-5},
        "temperature": {0: 303},
        "pressure": {0: 350000}}

# Pass the tear_guess to the SD tool
seq.set_guesses_for(m.fs.H101.inlet, tear_guesses)

Next, we need to tell the tool how to initialize a particular unit. We will be writing a python function which takes in a "unit" and calls the initialize method on that unit. 

In [37]:
def function(unit):
        unit.initialize(outlvl=1)

We are now ready to initialize our flowsheet in a sequential mode. Note that we specifically set the iteration limit to be 5 as we are trying to use this tool only to get a good set of initial values such that IPOPT can then take over and solve this flowsheet for us. 

In [38]:
seq.run(m, function)

2019-09-08 11:30:07 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.H101.control_volume.properties_in initialization
2019-09-08 11:30:07 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.H101.control_volume.properties_out initialization
2019-09-08 11:30:07 - INFO - idaes.core.unit_model - fs.H101 Initialization Step 1 Complete.
2019-09-08 11:30:07 - INFO - idaes.core.unit_model - fs.H101 Initialization Step 2 Complete.
2019-09-08 11:30:07 - INFO - idaes.core.unit_model - fs.H101 Initialization Complete.
2019-09-08 11:30:07 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.R101.control_volume.properties_in initialization
2019-09-08 11:30:07 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.R101.control_volume.properties_out initialization
2019-09-08 11:30:07 - INFO - idaes.core.unit_model - fs.R101 Initialization Step 1 Complete.
2019-09-08 11:30:07 - INFO - idae

2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.R101 Initialization Step 1 Complete.
2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.R101 Initialization Step 2 Complete.
2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.R101 Initialization Complete.
2019-09-08 11:30:09 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.F101.control_volume.properties_in initialization
2019-09-08 11:30:09 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.F101.control_volume.properties_out initialization
2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.F101 Initialization Step 1 Complete.
2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.F101 Initialization Step 2 Complete.
2019-09-08 11:30:09 - INFO - idaes.core.unit_model - fs.F101 Initialization Complete.
2019-09-08 11:30:09 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.S101.mixed_state initialization
2019-09-08 11:30:09 - 

2019-09-08 11:30:11 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.C101.control_volume.properties_in initialization
2019-09-08 11:30:11 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.C101.control_volume.properties_out initialization
2019-09-08 11:30:11 - INFO - idaes.core.unit_model - fs.C101 Initialization Step 1 Complete.
2019-09-08 11:30:11 - INFO - idaes.core.unit_model - fs.C101 Initialization Step 2 Complete.
2019-09-08 11:30:11 - INFO - idaes.core.unit_model - fs.C101 Initialization Complete.
2019-09-08 11:30:11 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.M101.toluene_feed_state initialization
2019-09-08 11:30:11 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.M101.hydrogen_feed_state initialization
2019-09-08 11:30:11 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.M101.vapor_recycle_state initialization
2019-

2019-09-08 11:30:12 - INFO - idaes.examples.properties.Workshop_Module_2.hda_ideal_VLE - Starting fs.F102.control_volume.properties_out initialization
2019-09-08 11:30:12 - INFO - idaes.core.unit_model - fs.F102 Initialization Step 1 Complete.
2019-09-08 11:30:12 - INFO - idaes.core.unit_model - fs.F102 Initialization Step 2 Complete.
2019-09-08 11:30:12 - INFO - idaes.core.unit_model - fs.F102 Initialization Complete.


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
We have now initialized the flowsheet. Let us run the flowsheet in a simulation mode to look at the results. To do this, complete the last line of code where we pass the model to the solver. You will need to type the following:
    
results = solver.solve(m, tee=True)

Use Shift+Enter to run the cell once you have typed in your code. 
</div>



In [40]:
# Create the solver object
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6, 'max_iter': 5000}

# Solve the model
results = solver.solve(m, tee=True)

Ipopt 3.12.8: tol=1e-06
max_iter=5000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma27.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      994
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      898

Total number of variables............................:      326
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      146
                     variables with only upper

 107  0.0000000e+00 1.04e+04 1.07e+15  -1.0 3.58e+03    -  1.00e+00 9.85e-04h  9
 108  0.0000000e+00 1.04e+04 1.07e+15  -1.0 6.37e+04    -  9.43e-01 9.85e-04h 10
 109  0.0000000e+00 1.04e+04 1.07e+15  -1.0 6.36e+04    -  1.00e+00 9.85e-04h 10
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  0.0000000e+00 1.04e+04 1.07e+15  -1.0 6.36e+04    -  9.45e-01 9.85e-04h 10
 111  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.35e+04    -  1.00e+00 9.85e-04h 10
 112  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.34e+04    -  9.47e-01 9.85e-04h 10
 113  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.34e+04    -  1.00e+00 9.85e-04h 10
 114  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.33e+04    -  9.49e-01 9.85e-04h 10
 115  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.32e+04    -  1.00e+00 9.85e-04h 10
 116  0.0000000e+00 1.03e+04 1.07e+15  -1.0 6.32e+04    -  9.51e-01 9.85e-04h 10
 117  0.0000000e+00 5.56e+05 1.09e+15  -1.0 6.31e+04    -  1.00e+00 5.04e-01w  1
 118  0.0000000e+00 2.54e+05

 204  0.0000000e+00 9.53e+03 1.08e+15  -1.0 5.15e+04    -  1.00e+00 1.96e-03h  9
 205  0.0000000e+00 9.51e+03 1.09e+15  -1.0 5.00e+04    -  1.00e+00 1.96e-03h  9
 206  0.0000000e+00 9.49e+03 1.09e+15  -1.0 4.81e+04    -  1.00e+00 1.96e-03h  9
 207  0.0000000e+00 9.47e+03 1.09e+15  -1.0 4.57e+04    -  1.00e+00 1.96e-03h  9
 208  0.0000000e+00 3.32e+05 1.21e+15  -1.0 4.24e+04    -  1.00e+00 5.01e-01w  1
 209  0.0000000e+00 1.68e+05 3.84e+15  -1.0 3.78e+04    -  5.05e-01 5.46e-01w  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 210  0.0000000e+00 9.10e+04 2.02e+18  -1.0 2.88e+03  -2.0 1.12e-01 9.81e-01h  1
 211  0.0000000e+00 9.10e+04 2.02e+18  -1.0 2.81e+06  -2.5 1.50e-03 4.32e-04f  2
 212  0.0000000e+00 9.10e+04 2.02e+18  -1.0 3.54e+06  -3.0 1.47e-03 2.20e-04f  3
 213  0.0000000e+00 9.10e+04 2.02e+18  -1.0 4.12e+06  -3.4 2.08e-03 2.30e-04f  3
 214  0.0000000e+00 9.10e+04 2.02e+18  -1.0 4.54e+06  -3.9 2.12e-03 1.30e-04f  4
 215  0.0000000e+00 9.10e+04

 304  0.0000000e+00 6.72e+03 1.67e+10  -1.0 3.57e+03    -  1.00e+00 2.11e-03h  9
 305  0.0000000e+00 6.71e+03 3.12e+10  -1.0 2.93e+03    -  1.00e+00 2.11e-03h  9
 306  0.0000000e+00 6.69e+03 5.77e+10  -1.0 2.05e+03    -  1.00e+00 2.11e-03h  9
 307  0.0000000e+00 6.68e+03 1.06e+11  -1.0 2.10e+03    -  1.00e+00 2.12e-03h  9
 308  0.0000000e+00 6.66e+03 1.63e+11  -1.0 4.82e+03    -  6.61e-01 2.13e-03h  9
 309  0.0000000e+00 6.65e+03 2.08e+11  -1.0 1.10e+04    -  3.54e-01 2.15e-03h  9
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310  0.0000000e+00 9.25e+03 1.50e+11  -1.0 1.38e+04    -  3.02e-01 5.52e-01w  1
 311  0.0000000e+00 7.29e+03 7.08e+12  -1.0 3.04e+03    -  2.54e-01 9.91e-01w  1
 312  0.0000000e+00 7.84e+03 6.96e+12  -1.0 2.07e+05    -  2.90e-02 3.48e-02w  1
 313  0.0000000e+00 6.63e+03 2.58e+11  -1.0 7.02e+03    -  3.02e-01 2.16e-03h  8
 314  0.0000000e+00 6.62e+03 3.17e+11  -1.0 1.62e+04    -  2.98e-01 2.16e-03h  9
 315  0.0000000e+00 6.61e+03

Objective...............:   0.0000000000000000e+00    0.0000000000000000e+00
Dual infeasibility......:   8.8079467308911410e-10    8.8079467308911410e-10
Constraint violation....:   2.9103830456733704e-11    8.1956386566162109e-08
Complementarity.........:   9.0909090909091298e-08    9.0909090909091298e-08
Overall NLP error.......:   7.3586953589023978e-09    9.0909090909091298e-08


Number of objective function evaluations             = 3210
Number of objective gradient evaluations             = 397
Number of equality constraint evaluations            = 3210
Number of inequality constraint evaluations          = 0
Number of equality constraint Jacobian evaluations   = 403
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 396
Total CPU secs in IPOPT (w/o function evaluations)   =      0.621
Total CPU secs in NLP function evaluations           =      0.092

EXIT: Optimal Solution Found.


In [41]:
# For testing purposes
from pyomo.environ import TerminationCondition
assert results.solver.termination_condition == TerminationCondition.optimal

Analyze the results of the square problem
-------------------------

What is the total operating cost? 

In [42]:
print('operating cost = $', value(m.fs.operating_cost))

operating cost = $ 404944.6501353814


For this operating cost, what is the amount of benzene we are able to produce and what purity we are able to achieve? 

In [43]:
m.fs.F102.report()

print()
print('benzene purity = ', value(m.fs.purity))


Unit : fs.F102                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key             : Value       : Fixed : Bounds
          Heat Duty :      3878.5 : False : (None, None)
    Pressure Change : -2.0000e+05 :  True : (None, None)

------------------------------------------------------------------------------------
    Stream Table
                                               Inlet    Vapor Outlet  Liquid Outlet
    flow_mol_phase_comp ('Liq', 'benzene')     0.18713   1.0000e-08       0.13938  
    flow_mol_phase_comp ('Liq', 'toluene')    0.081976   1.0000e-08      0.071799  
    flow_mol_phase_comp ('Liq', 'hydrogen') 2.6910e-07   1.0000e-08    2.1118e-07  
    flow_mol_phase_comp ('Liq', 'methane')  2.6910e-07   1.0000e-08    2.1118e-07  
    flow_mol_phase_comp ('Vap', 'benzene')  1.0000e-08     0.047743    1.0000e-08  
    flow_mol_phase

Next, let's look at how much benzene we are loosing with the light gases out of F101. IDAES has tools for creating stream tables based on the `Arcs` and/or `Ports` in a flowsheet. Let us create and print a simple stream table showing the stream leaving the reactor and the vapor stream from F101.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
How much benzene are we losing in the F101 vapor outlet stream?
</div>


In [44]:
from idaes.core.util.tables import create_stream_table_dataframe, stream_table_dataframe_to_string

st = create_stream_table_dataframe({"Reactor": m.fs.s05, "Light Gases": m.fs.s06})
st

,Reactor,Light Gases
"flow_mol_phase_comp ('Liq', 'benzene')",0.167639,1.000000e-08
"flow_mol_phase_comp ('Liq', 'toluene')",0.023130,1.000000e-08
"flow_mol_phase_comp ('Liq', 'hydrogen')",0.207166,1.000000e-08
"flow_mol_phase_comp ('Liq', 'methane')",0.487846,1.000000e-08
"flow_mol_phase_comp ('Vap', 'benzene')",0.154129,1.346430e-01
"flow_mol_phase_comp ('Vap', 'toluene')",0.079048,2.020218e-02
"flow_mol_phase_comp ('Vap', 'hydrogen')",0.222913,4.300786e-01
"flow_mol_phase_comp ('Vap', 'methane')",0.682373,1.170219e+00
temperature,830.063048,3.250000e+02
pressure,349999.999191,3.500000e+05


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
You can querry additional variables here if you like. 

Use Shift+Enter to run the cell once you have typed in your code. 
</div>
